In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Config

# Load the tokenizer and configuration
tokenizer = GPT2Tokenizer.from_pretrained('model_tokenizer')
config = GPT2Config.from_pretrained('model_tokenizer')

# Initialize the model
model = GPT2ForSequenceClassification.from_pretrained('model_tokenizer', config=config)

# Load the trained model weights
model.load_state_dict(torch.load('gpt2_sentiment_analysis.pth'))
model.eval()  # Set the model to evaluation mode
model.to(torch.device("cuda" if torch.cuda.is available() else "cpu"))

# Prediction function
def predict_headline(headline, tokenizer, model, device):
    model.eval()  # Ensure the model is in evaluation mode
    inputs = tokenizer.encode_plus(
        headline, 
        return_tensors="pt", 
        max_length=512, 
        truncation=True, 
        padding="max_length"
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=-1)

    return "Increase" if prediction == 1 else "Decrease"

# Example usage
headline = "Stock market hits new record high as investor confidence soars."
print(predict_headline(headline, tokenizer, model, torch.device("cpu")))
